# Working with the FERC Form 1 Extract / Transform
This notebook steps through PUDL's extract and transform steps for FERC Form 1 to make it easier to test and add new years of data, or new tables from the various spreadsheets that haven't been integrated yet.

In [233]:
%load_ext autoreload
%autoreload 3
import pudl
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [234]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [235]:
pudl_settings = pudl.workspace.setup.get_defaults()

## Pick the tables you want to load

In [236]:
# tables = [
#     "balance_sheet_assets_ferc1",
#     "balance_sheet_liabilities_ferc1",
#     "depreciation_amortization_summary_ferc1",
#     "electric_energy_dispositions_ferc1",
#     "electric_energy_sources_ferc1",
#     "electric_opex_ferc1",
#     "electric_plant_depreciation_changes_ferc1",
#     "fuel_ferc1",
#     "income_statement_ferc1",
#     "plants_hydro_ferc1",
#     "plants_pumped_storage_ferc1",
#     "plants_small_ferc1",
#     "plants_steam_ferc1",
#     "plant_in_service_ferc1",
#     "purchased_power_ferc1",
#     "retained_earnings_ferc1",
#     "transmission_statistics_ferc1",
#     "utility_plant_summary_ferc1",
#     "electricity_sales_by_rate_schedule_ferc1"
# ]

#tables = ["electricity_sales_by_rate_schedule_ferc1", "income_statement_ferc1", "balance_sheet_assets_ferc1"]
tables = ["plants_steam_ferc1", "fuel_ferc1"]

#table = "electricity_sales_by_rate_schedule_ferc1"

In [237]:
ferc1_settings = pudl.settings.Ferc1Settings(tables=tables)

## Extract DBF and XBRL Data:

In [ ]:
# Extract old FERC form 1 data from DBF (2020 -)
ferc1_dbf_raw_dfs = pudl.extract.ferc1.extract_dbf(
    ferc1_settings=ferc1_settings, pudl_settings=pudl_settings
)
# Extract new FERC form 1 data from XBRL (2021 + )
ferc1_xbrl_raw_dfs = pudl.extract.ferc1.extract_xbrl(
    ferc1_settings=ferc1_settings, pudl_settings=pudl_settings
)
# Extract XBRL metadata
xbrl_metadata_json_dict = pudl.extract.ferc1.extract_xbrl_metadata(
    ferc1_settings=ferc1_settings, pudl_settings=pudl_settings
)
#xbrl_metadata_json_dict = {table: pudl.extract.ferc1.extract_xbrl_metadata(ferc1_settings, pudl_settings)[table] for table in tables}
xbrl_metadata_json_dict = pudl.extract.ferc1.extract_xbrl_metadata(
    ferc1_settings=ferc1_settings, pudl_settings=pudl_settings
)

In [ ]:
ferc1_xbrl_raw_dfs[table]

## Transform FERC 1 Tables:

In [ ]:
from pudl.transform.ferc1 import *
from pudl.transform.params import *

transformers = [
    bsa := BalanceSheetAssetsFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["balance_sheet_assets_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    bsl := BalanceSheetLiabilitiesFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["balance_sheet_liabilities_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    das := DepreciationAmortizationSummaryFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["depreciation_amortization_summary_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    eed := ElectricEnergyDispositionsFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["electric_energy_dispositions_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ees := ElectricEnergySourcesFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["electric_energy_sources_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    eo := ElectricOpexFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["electric_opex_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    epdc := ElectricPlantDepreciationChangesFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["electric_plant_depreciation_changes_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ff := FuelFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["fuel_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ins := IncomeStatementFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["income_statement_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ph := PlantsHydroFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plants_hydro_ferc1"], 
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    pps := PlantsPumpedStorageFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plants_pumped_storage_ferc1"], 
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    psm := PlantsSmallFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plants_small_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    pst := PlantsSteamFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plants_steam_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    pis := PlantInServiceFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plant_in_service_ferc1"], 
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    pp := PurchasedPowerFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["purchased_power_ferc1"], 
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    re := RetainedEarningsFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["retained_earnings_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ts := TransmissionStatisticsFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["transmission_statistics_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    ups := UtilityPlantSummaryFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["utility_plant_summary_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    ),
    esbrs := ElectricitySalesByRateScheduleFerc1TableTransformer(
    #xbrl_metadata_json=xbrl_metadata_json_dict["electricity_sales_by_rate_schedule_ferc1"],
    cache_dfs=True, 
    clear_cached_dfs=False
    ),
]

In [257]:
from pudl.transform.ferc1 import *
from pudl.transform.params import *

ff = FuelFerc1TableTransformer(
    xbrl_metadata_json=xbrl_metadata_json_dict["fuel_ferc1"],
    cache_dfs=True, 
    clear_cached_dfs=False
)

pst = PlantsSteamFerc1TableTransformer(
        xbrl_metadata_json=xbrl_metadata_json_dict["plants_steam_ferc1"],
        cache_dfs=True, 
        clear_cached_dfs=False
    )



2023-01-31 16:04:53 [    INFO] catalystcoop.pudl.transform.ferc1:896 fuel_ferc1: Processing XBRL metadata.


fuel_ferc1: Processing XBRL metadata.


2023-01-31 16:04:53 [    INFO] catalystcoop.pudl.transform.ferc1:896 plants_steam_ferc1: Processing XBRL metadata.


plants_steam_ferc1: Processing XBRL metadata.


### Transform Individual Tables

In [258]:
# Pick one table to transform
TRANSFORMER = ff
table = TRANSFORMER.table_id.value

In [259]:
#for transformer in transformers:

bb = TRANSFORMER.process_xbrl(
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[table]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[table]["duration"]
)

2023-01-31 16:06:49 [    INFO] catalystcoop.pudl.transform.classes:1194 fuel_ferc1: Attempting to rename 0 columns.


fuel_ferc1: Attempting to rename 0 columns.


2023-01-31 16:06:49 [    INFO] catalystcoop.pudl.transform.ferc1:1056 fuel_ferc1: Unstacking balances to the report years.


fuel_ferc1: Unstacking balances to the report years.


2023-01-31 16:06:49 [    INFO] catalystcoop.pudl.transform.classes:1194 fuel_ferc1: Attempting to rename 0 columns.


...

fuel_ferc1: Categorizing string columns using a controlled vocabulary.


2023-01-31 16:06:49 [    INFO] catalystcoop.pudl.transform.ferc1:1663 fuel_ferc1: Aggregating 30 rows with duplicate primary keys out of 1312 total rows.


fuel_ferc1: Aggregating 30 rows with duplicate primary keys out of 1312 total rows.


2023-01-31 16:06:49 [    INFO] catalystcoop.pudl.transform.ferc1:1667 fuel_ferc1: Dropping 98 records with inconsistent fuel units preventing aggregation out of 1312 total rows.


fuel_ferc1: Dropping 98 records with inconsistent fuel units preventing aggregation out of 1312 total rows.


In [260]:
# Process steam
TRANSFORMER = pst
#print(TRANSFORMER)
#table = TRANSFORMER.table_id.value

steam = TRANSFORMER.process_xbrl(
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[table]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[table]["duration"]
)

2023-01-31 16:06:53 [    INFO] catalystcoop.pudl.transform.ferc1:1003 plants_steam_ferc1: Processing XBRL data pre-concatenation.


plants_steam_ferc1: Processing XBRL data pre-concatenation.


2023-01-31 16:06:53 [    INFO] catalystcoop.pudl.transform.classes:1194 plants_steam_ferc1: Attempting to rename 0 columns.


plants_steam_ferc1: Attempting to rename 0 columns.


2023-01-31 16:06:53 [    INFO] catalystcoop.pudl.transform.ferc1:1056 plants_steam_ferc1: Unstacking balances to the report years.


...

plants_steam_ferc1: Attempting to rename 43 columns.


2023-01-31 16:06:53 [ WARNING] catalystcoop.pudl.transform.classes:1203 plants_steam_ferc1: Attempting to rename columns which are not present in the dataframe.
Missing columns: {'net_continuous_plant_capability', 'net_peak_demand_on_plant', 'maintenance_of_boiler_plant_steam_power_generation', 'year_plant_originally_constructed', 'net_generation_excluding_plant_use', 'plant_average_number_of_employees', 'plant_hours_connected_to_load', 'electric_expenses_steam_power_generation', 'miscellaneous_steam_power_expenses', 'net_continuous_plant_capability_not_limited_by_condenser_water', 'maintenance_of_electric_plant_steam_power_generation', 'plant_name', 'asset_retirement_costs_steam_production', 'coolants_and_water', 'cost_per_kilowatt_of_installed_capacity', 'plant_construction_type', 'expenses_per_net_kilowatt_hour', 'maintenance_supervision_and_engineering_steam_power_generation', 'power_production_expenses_steam_power', 'steam_expenses_steam_power_generation', 'allowances', 'maintenan

plants_steam_ferc1: Attempting to rename columns which are not present in the dataframe.
Missing columns: {'net_continuous_plant_capability', 'net_peak_demand_on_plant', 'maintenance_of_boiler_plant_steam_power_generation', 'year_plant_originally_constructed', 'net_generation_excluding_plant_use', 'plant_average_number_of_employees', 'plant_hours_connected_to_load', 'electric_expenses_steam_power_generation', 'miscellaneous_steam_power_expenses', 'net_continuous_plant_capability_not_limited_by_condenser_water', 'maintenance_of_electric_plant_steam_power_generation', 'plant_name', 'asset_retirement_costs_steam_production', 'coolants_and_water', 'cost_per_kilowatt_of_installed_capacity', 'plant_construction_type', 'expenses_per_net_kilowatt_hour', 'maintenance_supervision_and_engineering_steam_power_generation', 'power_production_expenses_steam_power', 'steam_expenses_steam_power_generation', 'allowances', 'maintenance_of_miscellaneous_steam_plant', 'net_continuous_plant_capability_limit

2023-01-31 16:06:53 [ WARNING] catalystcoop.pudl.transform.ferc1:1387 plants_steam_ferc1: Found 500 duplicate record_ids: 
['steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_baxter_wilson'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_gerald_andrus'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_independence'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001330_unit_4'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001330_units_13'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c000906_fort_martin'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c000906_harrison'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_glenwood_steam'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_port_jefferson_steam'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_nort

plants_steam_ferc1: Found 500 duplicate record_ids: 
['steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_baxter_wilson'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_gerald_andrus'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c008999_independence'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001330_unit_4'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001330_units_13'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c000906_fort_martin'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c000906_harrison'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_glenwood_steam'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_port_jefferson_steam'
 'steam_electric_generating_plant_statistics_large_plants_402_2021_c001322_northport_steam'
 'steam_electric_generating_plant_statistics_large_plants

In [261]:
# View missing plant names
steam[steam['plant_name_ferc1'] == "0"]

,utility_id_ferc1_xbrl,start_date,end_date,fuel_kind_axis,plant_name_ferc1,average_cost_of_fuel_burned_per_kilowatt_hour_net_generation,average_cost_of_fuel_per_unit_burned,average_cost_of_fuel_burned_per_million_british_thermal_unit,fuel_burned_average_heat_content,fuel_kind,average_british_thermal_unit_per_kilowatt_hour_net_generation,quantity_of_fuel_burned,fuel_unit,average_cost_of_fuel_per_unit_as_delivered,report_year,record_id,utility_id_ferc1
1180,C011304,2021-01-01,2021-12-31,COAL,0,0.027,53.876,236.732,11517.0,COAL,10800.0,2357771.0,T,59.736,2021,steam_electric_generating_plant_statistics_lar...,444
1181,C011304,2021-01-01,2021-12-31,OIL,0,0.000,2.167,1367.210,136000.0,OIL,0.0,844331.0,bbl,2.276,2021,steam_electric_generating_plant_statistics_lar...,444
1231,C000447,2021-01-01,2021-12-31,Gas,0,0.000,0.000,0.000,NaN,Gas,0.0,NaN,Mcf,0.000,2021,steam_electric_generating_plant_statistics_lar...,185


#### Transform Step-by-Step

In [262]:
start = TRANSFORMER.transform_start(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMER.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["duration"]
)

2023-01-31 16:07:02 [    INFO] catalystcoop.pudl.transform.ferc1:982 plants_steam_ferc1: Processing DBF data pre-concatenation.


plants_steam_ferc1: Processing DBF data pre-concatenation.


2023-01-31 16:07:02 [    INFO] catalystcoop.pudl.transform.classes:1194 plants_steam_ferc1: Attempting to rename 43 columns.


plants_steam_ferc1: Attempting to rename 43 columns.


2023-01-31 16:07:03 [    INFO] catalystcoop.pudl.transform.ferc1:1003 plants_steam_ferc1: Processing XBRL data pre-concatenation.


...

plants_steam_ferc1: Combining XBRL instant & duration tables using RIGHT-MERGE.


2023-01-31 16:07:03 [    INFO] catalystcoop.pudl.transform.classes:1194 plants_steam_ferc1: Attempting to rename 43 columns.


plants_steam_ferc1: Attempting to rename 43 columns.


2023-01-31 16:07:03 [    INFO] catalystcoop.pudl.transform.ferc1:819 plants_steam_ferc1: Concatenating DBF + XBRL dataframes.


plants_steam_ferc1: Concatenating DBF + XBRL dataframes.


In [263]:
start[start['record_id']=='f1_steam_1999_12_72_0_1']

,utility_id_ferc1_dbf,report_year,plant_name_ferc1,plant_type,construction_type,construction_year,installation_year,capacity_mw,peak_demand_mw,plant_hours_connected_while_generating,plant_capability_mw,not_water_limited_capacity_mw,water_limited_capacity_mw,avg_num_employees,net_generation_kwh,capex_land,capex_structures,capex_equipment,capex_total,capex_per_kw,opex_operations,opex_fuel,opex_coolants,opex_steam,opex_steam_other,opex_transfer,opex_electric,opex_misc_power,opex_rents,opex_allowances,opex_engineering,opex_structures,opex_boiler,opex_plants,opex_misc_steam,opex_production_total,opex_per_kwh,asset_retirement_cost,record_id,utility_id_ferc1,utility_id_ferc1_xbrl,date,start_date,end_date,plant_name,order_number
9645,72.0,1999,,Steam,Conventional,1955,1956,1303.56,1327.0,8760.0,NaN,1284.0,NaN,390.0,8.531868e+09,673317.0,64561620.0,322548092.0,387783029.0,297.48,1154409.0,107221337.0,NaN,4377325.0,NaN,NaN,2030686.0,4309783.0,2400.0,NaN,425959.0,1277712.0,11216023.0,2529112.0,709251.0,135253997.0,0.0159,NaN,f1_steam_1999_12_72_0_1,409,NaN,NaN,NaT,NaT,NaN,NaN


In [264]:
# Transform fuel table for use in 

TRANSFORMERF = ff
tablef = TRANSFORMERF.table_id.value

fuel = TRANSFORMERF.transform(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMERF.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMERF.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMERF.table_id.value]["duration"]
)

2023-01-31 16:07:12 [    INFO] catalystcoop.pudl.transform.ferc1:982 fuel_ferc1: Processing DBF data pre-concatenation.


fuel_ferc1: Processing DBF data pre-concatenation.


2023-01-31 16:07:12 [    INFO] catalystcoop.pudl.transform.classes:1194 fuel_ferc1: Attempting to rename 17 columns.


fuel_ferc1: Attempting to rename 17 columns.


2023-01-31 16:07:13 [ WARNING] catalystcoop.pudl.transform.ferc1:1387 fuel_ferc1: Found 1 duplicate record_ids: 
['f1_fuel_2000_12_24_3_1'].


...

Correcting units of fuel_cost_per_mmbtu where fuel_type_code_pudl==oil.


2023-01-31 16:07:15 [    INFO] catalystcoop.pudl.transform.classes:719 8083/17623 (45.87%) of records could not be corrected and were set to NA.


8083/17623 (45.87%) of records could not be corrected and were set to NA.


2023-01-31 16:07:15 [    INFO] catalystcoop.pudl.transform.classes:1327 fuel_ferc1: Enforcing database schema on dataframe.


fuel_ferc1: Enforcing database schema on dataframe.


In [266]:
main = TRANSFORMER.transform_main(
    start, transformed_fuel = fuel
)

2023-01-31 16:09:28 [    INFO] catalystcoop.pudl.transform.classes:1218 plants_steam_ferc1: Normalizing freeform string columns.


plants_steam_ferc1: Normalizing freeform string columns.


2023-01-31 16:09:28 [    INFO] catalystcoop.pudl.transform.classes:1242 plants_steam_ferc1: Categorizing string columns using a controlled vocabulary.


plants_steam_ferc1: Categorizing string columns using a controlled vocabulary.


2023-01-31 16:09:28 [    INFO] catalystcoop.pudl.transform.classes:1267 plants_steam_ferc1: Converting units and renaming columns accordingly.


...

Found report_year=2021 2 times in plant_id_ferc1=1281


2023-01-31 16:28:26 [   ERROR] catalystcoop.pudl.analysis.classify_plants_ferc1:594 Found report_year=2004 2 times in plant_id_ferc1=1567


Found report_year=2004 2 times in plant_id_ferc1=1567


2023-01-31 16:28:26 [   ERROR] catalystcoop.pudl.analysis.classify_plants_ferc1:594 Found report_year=2005 2 times in plant_id_ferc1=1567


Found report_year=2005 2 times in plant_id_ferc1=1567


In [269]:
# Test it works!
main[main['record_id'].str.contains('f1_steam_1999_12_72_0_1')] #clifty creek
main[main['record_id'].str.contains('f1_steam_1998_12_64_0_1')] #hardee power station


,utility_id_ferc1_dbf,report_year,capacity_mw,peak_demand_mw,plant_hours_connected_while_generating,plant_capability_mw,not_water_limited_capacity_mw,water_limited_capacity_mw,avg_num_employees,capex_land,capex_structures,capex_equipment,capex_total,opex_operations,opex_fuel,opex_coolants,opex_steam,opex_steam_other,opex_transfer,opex_electric,opex_misc_power,opex_rents,opex_allowances,opex_engineering,opex_structures,opex_boiler,opex_plants,opex_misc_steam,opex_production_total,asset_retirement_cost,record_id,utility_id_ferc1,utility_id_ferc1_xbrl,date,start_date,end_date,plant_name,order_number,plant_name_ferc1,construction_type,plant_type,capex_per_mw,opex_per_mwh,net_generation_mwh,construction_year,installation_year,plant_id_ferc1
6370,64.0,1998,349.0,313.0,4649.0,295.0,295.0,NaN,NaN,NaN,30484465.0,176340418.0,206824883.0,145058.0,21212847.0,NaN,NaN,NaN,NaN,1429184.0,NaN,212211.0,NaN,141832.0,149465.0,NaN,699439.0,NaN,23990036.0,NaN,f1_steam_1998_12_64_0_1,253,NaN,NaN,NaT,NaT,NaN,NaN,hardee power station,outdoor,combustion_turbine,592621.4,26.2,916579.0,1992.0,1992.0,249


In [203]:
start.loc[start['record_id']=="f1_steam_2001_12_204_0_1", 'plant_name_ferc1'] = "TEST"
start.loc[start['record_id']=="f1_steam_2001_12_204_0_1", 'plant_name_ferc1']

14375    TEST
Name: plant_name_ferc1, dtype: object

In [213]:
df = start
for i in SpotFixValues['spot_fix']:
    idcol = list(i.keys())[0] # Get name of identifying column
    
    if idcol in list(df.columns): # For each row to be spot-fixed
        for key in i['fixes']: # For each fix
            if key in list(df.columns):
                df.loc[df[idcol]==i[idcol], key] = i['fixes'][key] # Manually update value
    else:
        # Could log error here
        pass

print(df.loc[df['plant_name_ferc1']=='seabrook'])


       utility_id_ferc1_dbf  report_year plant_name_ferc1 plant_type construction_type construction_year installation_year  capacity_mw  peak_demand_mw  plant_hours_connected_while_generating  plant_capability_mw  not_water_limited_capacity_mw  water_limited_capacity_mw  avg_num_employees  net_generation_kwh  capex_land  capex_structures  capex_equipment  capex_total  capex_per_kw  opex_operations  opex_fuel  opex_coolants  opex_steam  opex_steam_other  opex_transfer  opex_electric  opex_misc_power  opex_rents  opex_allowances  opex_engineering  opex_structures  opex_boiler  opex_plants  opex_misc_steam  opex_production_total  opex_per_kwh  asset_retirement_cost                 record_id  utility_id_ferc1 utility_id_ferc1_xbrl date start_date end_date plant_name  order_number
11185                 204.0         2000         seabrook    Nuclear   Fully contained              1990              1990         26.0            25.0                                   150.0                  NaN 

In [ ]:
def spot_fix_values(self, df: pd.DataFrame, params: SpotFixValues) -> pd.DataFrame:
        """Manually fix one-off singular missing values.

        With our new ferc1 transform process we are less intense about dropping records.
        Because of that, a fair amount of records have "" or 0 as a plant name. 
        Most of these records have no other data points in them and thus can be dropped.
        But some of them actually have some information. Manual investigation of some
        of these records led to some pretty easy identification of plant names. This
        function takes a dictionary of these fixes and applies them to the dataframe.

        There are probably plenty of other spot fixes one could add here.

        Params:
            df: Pre-processed, concatenated XBRL and DBF data.
            params: an instance of :class:`SpotFixValues`

        Returns:
            The same input DataFrame but with some spot fixes corrected.
        """
        logger.info(f"{self.table_id.value}: Spot fixing some values")
        
        
        # Define rows and columns to change
        for i in params:
            idcol = list(i.keys())[0] # Get name of identifying column
    
            if idcol in list(df.columns): # For each row to be spot-fixed
                for key in i['fixes']: # For each fix
                    if key in list(df.columns):
                        df.loc[df[idcol]==i[idcol], key] = i['fixes'][key] # Manually update value
            else:
                # Could log error here
                pass

        return df

In [97]:
end = TRANSFORMER.transform_end(
    main
)
end[end['record_id']=='f1_steam_2014_12_276_0_1']

2023-01-31 10:20:15 [    INFO] catalystcoop.pudl.transform.classes:1264 plants_steam_ferc1: Enforcing database schema on dataframe.


plants_steam_ferc1: Enforcing database schema on dataframe.


KeyError: 'record_id_ferc1'

In [104]:
end[end['plant_name_ferc1']=='0']

,record_id,utility_id_ferc1,report_year,plant_id_ferc1,plant_name_ferc1,plant_type,construction_type,construction_year,installation_year,capacity_mw,peak_demand_mw,plant_hours_connected_while_generating,plant_capability_mw,water_limited_capacity_mw,not_water_limited_capacity_mw,avg_num_employees,net_generation_mwh,capex_land,capex_structures,capex_equipment,capex_total,capex_per_mw,opex_operations,opex_fuel,opex_coolants,opex_steam,opex_steam_other,opex_transfer,opex_electric,opex_misc_power,opex_rents,opex_allowances,opex_engineering,opex_structures,opex_boiler,opex_plants,opex_misc_steam,opex_production_total,opex_per_mwh,asset_retirement_cost


#### Transform All Steps Together

In [35]:
full = TRANSFORMER.transform(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMER.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["duration"]
)

2023-01-27 11:07:05 [    INFO] catalystcoop.pudl.transform.ferc1:981 fuel_ferc1: Processing DBF data pre-concatenation.


fuel_ferc1: Processing DBF data pre-concatenation.


2023-01-27 11:07:06 [    INFO] catalystcoop.pudl.transform.classes:1144 fuel_ferc1: Attempting to rename 17 columns.


fuel_ferc1: Attempting to rename 17 columns.


2023-01-27 11:07:06 [ WARNING] catalystcoop.pudl.transform.ferc1:1386 fuel_ferc1: Found 1 duplicate record_ids: 
['f1_fuel_2000_12_24_3_1'].


...

Correcting units of fuel_cost_per_mmbtu where fuel_type_code_pudl==oil.


2023-01-27 11:07:07 [    INFO] catalystcoop.pudl.transform.classes:719 8083/17623 (45.87%) of records could not be corrected and were set to NA.


8083/17623 (45.87%) of records could not be corrected and were set to NA.


2023-01-27 11:07:07 [    INFO] catalystcoop.pudl.transform.classes:1264 fuel_ferc1: Enforcing database schema on dataframe.


fuel_ferc1: Enforcing database schema on dataframe.


In [65]:
len(full)

48841

### Transform All Tables

In [8]:
transformed_tables = {}

for transformer in transformers:
    transformed_tables[transformer.table_id.value] = transformer.transform(
        raw_dbf=ferc1_dbf_raw_dfs[transformer.table_id.value],
        raw_xbrl_instant=ferc1_xbrl_raw_dfs[transformer.table_id.value]["instant"],
        raw_xbrl_duration=ferc1_xbrl_raw_dfs[transformer.table_id.value]["duration"]
    )

2023-01-27 16:22:15 [    INFO] catalystcoop.pudl.transform.ferc1:1072 balance_sheet_assets_ferc1: Processing DBF data pre-concatenation.


balance_sheet_assets_ferc1: Processing DBF data pre-concatenation.


2023-01-27 16:22:15 [    INFO] catalystcoop.pudl.transform.ferc1:1135 balance_sheet_assets_ferc1: After selection only annual records, we have 40.6% of the original table.


balance_sheet_assets_ferc1: After selection only annual records, we have 40.6% of the original table.


2023-01-27 16:22:15 [    INFO] catalystcoop.pudl.transform.ferc1:348 Aligning row numbers from DBF row to XBRL map for ['f1_comp_balance_db']


...

2023-01-27 16:22:38 [    INFO] catalystcoop.pudl.transform.classes:823 0.4% of records (67 rows) contain only {'', '0', nan, '-', 'not applicable', <NA>} values in required columns. Dropped these 💩💩💩 records.


0.4% of records (67 rows) contain only {'', '0', nan, '-', 'not applicable', <NA>} values in required columns. Dropped these 💩💩💩 records.


2023-01-27 16:22:38 [    INFO] catalystcoop.pudl.transform.classes:1264 plants_small_ferc1: Enforcing database schema on dataframe.


plants_small_ferc1: Enforcing database schema on dataframe.


TypeError: PlantsSteamFerc1TableTransformer.transform() missing 1 required positional argument: 'transformed_fuel'

In [ ]:
transformed_table